<a href="https://colab.research.google.com/github/gduncan2/CS445_OpticalFlowProj/blob/main/445FinalProj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ffmpeg -hwaccel cuda -i hogwarts_legacy_staircase_video.mp4 -filter:v fps=24 -c:v hevc_nvenc output_24_fps_nvenc_hevc.mp4
# I first used ffmpeg with CUDA to convert the video to 24 fps, from the original choppy 30 fps gameplay, to keep the input rate constant.
# The original video is "hogwarts_legacy_staircase_video.mp4" and the output is "output_24_fps_nvenc_hevc.mp4".

In [ ]:
# Now we can split this video into frames using ffmpeg and CUDA.
# ffmpeg -hwaccel cuda -i output_24_fps_nvenc_hevc.mp4 -vsync 0 frames/frame_%05d.png